In [1]:
## import all necessary packages and functions.
import csv # read and write csv files
import json
import pandas as pd
import numpy as np
from datetime import datetime # operations to parse dates
from pprint import pprint # use to print data structures like dictionaries in
                          # a nicer way than the base print function.
#pd.options.display.float_format = '{:,.0f}'.format

In [19]:
def to_canon(myfile):
    """
    Esta funcion recibe una o varias entradas de datos. 
    Si es una directamente trabaja con ella para convertirla a modelo a canonico.
    Si son varias, entendiendose que desde la fuente el mes se fractura en varias partes, las junta tras la conversion.
    
    format_file admite 1=csv o 2=json, da error para otro valor
    
    format_date especifica el formato de la fecha, valores posibles 
        1=de numero a fecha(lon), 2=juntar fecha y hora(mex), 3=coreano, 4=de numero a fecha 2 (NY), 
        5=de cadena json a fecha(mad)
        
    calc_duration calcula la duracion si se necesita, 1 para todo menos 2 para londres que la da directa
    
    age 1=lleva edad ya formateada (mex), 2=lleva nacimiento (NY) y -9 si no hay edad
    
    gender 1=tiene genero, 2=tiene pero M y F en lugar de numeros (mex) y -9 si no hay genero
    
    usertype 1=si el dataset tiene tipos de usuario, -9 si no tiene
    
    city me dice que ciudad le estoy pasando
    10=madrid,  20=new york,  30=mexico,  40=seoul,  50=london,  60=buenos aires,  70=chicago
    
    """
    
    campos = pd.read_csv(myfile)

    listavalores = campos['valores'].tolist()

    source=listavalores[0]
    destination=listavalores[1]
    format_file=int(listavalores[2])
    year=int(listavalores[3])
    month=int(listavalores[4])
    format_date=int(listavalores[5])
    calc_duration=int(listavalores[6])
    age=int(listavalores[7])
    gender=int(listavalores[8])
    usertype=int(listavalores[9])
    city=int(listavalores[10])

    
    if format_file==2:
        df=pd.read_json(source, encoding='unicode_escape',lines=True)
        if city==10:
            df=df.drop({'_id','idplug_base','idunplug_base','idunplug_station','idplug_station'}, axis=1)
    
    elif source.find('$')!=-1:
        sources=source.split('$')
        df=pd.DataFrame()
        for i in range(len(sources)):
            dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
            df=df.append(dfaux)
        
    elif source.find('$')==-1:
        df=pd.read_csv(source,encoding='unicode_escape',low_memory=False)
        
    else:
        print("Formato de archivo no soportado")

    
    #Problema compatibilidad Seoul
    #df=df.sort_values([df.columns[5]])
    #df=df.iloc[20:]
    
    if format_date==1:
        df['starttime']=pd.to_datetime(df['Start Date'], dayfirst = True)
        df['stoptime']=pd.to_datetime(df['End Date'], dayfirst = True)
    elif format_date==2:
        df['starttime']=pd.to_datetime(df['Fecha_Retiro']+' '+df['Hora_Retiro'], dayfirst = True)
        df['stoptime']=pd.to_datetime(df['Fecha_Arribo']+' '+df['Hora_Arribo'], dayfirst = True)
    elif format_date==3:
        
        df['starttime']=pd.to_datetime(df[df.columns[1]],errors='coerce', dayfirst = True)
        df['stoptime']=pd.to_datetime(df[df.columns[5]],errors='coerce', dayfirst = True)
        #seoul 2019
        #df['starttime']=pd.to_datetime(df['´ë¿©ÀÏ½Ã'], dayfirst = True)
        #df['stoptime']=pd.to_datetime(df['¹Ý³³ÀÏ½Ã'], dayfirst = True)
    elif format_date==4:
        df['starttime']=pd.to_datetime(df['starttime']).apply(lambda x: x.replace(microsecond=0))
        df['stoptime']=pd.to_datetime(df['stoptime']).apply(lambda x: x.replace(microsecond=0))
    elif format_date==5:
        #MADRID, para los datos antiguos, hasta julio
        df['unplug_hourTime'] = (df['unplug_hourTime'].astype(str).str[11:21]+' '+df['unplug_hourTime'].astype(str).str[22:34])
        #para los datos a partir de julio, cambian el formato
        #df['unplug_hourTime'] = (df['unplug_hourTime'].astype(str).str[0:10]+' '+df['unplug_hourTime'].astype(str).str[11:18])
        df['starttime']=pd.to_datetime(df['unplug_hourTime'])
        df['stoptime']=pd.to_datetime(df['unplug_hourTime'])
    elif format_date==6:
        df['starttime']=pd.to_datetime(df['fecha_origen_recorrido'])
        df['stoptime']=pd.to_datetime(df['fecha_destino_recorrido'])
    elif format_date==7:
        #pre 2018 q1
        #df['starttime']=pd.to_datetime(df['01 - Rental Details Local Start Time'])
        #df['stoptime']=pd.to_datetime(df['01 - Rental Details Local End Time'])
        #else
        df['starttime']=pd.to_datetime(df['start_time'])
        df['stoptime']=pd.to_datetime(df['end_time'])
    else:
        print('Formato no valido, calculo de fecha y hora no soportado')
        
    df = df[df['starttime'].dt.month == month]
    
    df['Start Dates']=pd.to_datetime(df['starttime']).dt.date
    df['Start Time']=pd.to_datetime(df['starttime']).dt.time
    df['End Dates']=pd.to_datetime(df['stoptime']).dt.date
    df['End Time']=pd.to_datetime(df['stoptime']).dt.time
    
    if calc_duration==0:
        #Se entiende que no hay que calcular la duracion
        df['Duration']=df['Duration']
    
    elif calc_duration==1:
        df['tripduration'] = (df['stoptime'] - df['starttime'])
        df['Duration'] = df['tripduration'].dt.total_seconds()
        df.dropna(subset = ['Duration'], inplace=True)
        df['Duration'] = df['Duration'].astype(int)
    else:
        print('Formato no valido, calculo de duracion no soportado')    
    
    if gender==1:
        #Se entiende que el genero ya viene formateado
        df['gender']=df['gender']
    elif gender==2:
        df.genero_usuario=df.genero_usuario.apply(lambda y:1 if y=="M" else 2)
        #BA pero solo 2019 a partir de mayo o junio
        #df.genero_usuario=df.genero_usuario.apply(lambda y:1 if y=="MASCULINO" else y)
        #df.genero_usuario=df.genero_usuario.apply(lambda y:2 if y=="FEMENINO" else y)
        #df.genero_usuario=df.genero_usuario.apply(lambda y:0 if y=="NO INFORMADO" else y)
        #df['genero_usuario']=df['genero_usuario'].fillna(-99)
    elif gender==3:
        df.Genero_Usuario=df.Genero_Usuario.apply(lambda y:1 if y=="M" else 2)
    elif gender==4:
        #pre 2018 q1
        #df['Member Gender']=df['Member Gender'].apply(lambda y:1 if y=="Male" else y)
        #df['Member Gender']=df['Member Gender'].apply(lambda y:2 if y=="Female" else y)
        #df['gender']=df['Member Gender'].fillna(-99)
        #else
        df.gender=df.gender.apply(lambda y:1 if y=="Male" else y)
        df.gender=df.gender.apply(lambda y:2 if y=="Female" else y)
        df['gender']=df['gender'].fillna(-99)
    elif gender==-9:
        df['Gender'] = '-99'
    else:
        print('Formato no valido, calculo de genero no soportado')
        
        
    d = {range(10, 16): 1, range(17, 18): 2, range(19, 26): 3, range(27, 40): 4, range(41, 65): 5, range(66, 100): 6}

    if age==1:
        df.ageRange=df.ageRange.apply(lambda y:-99 if y=="0" else y)
    elif age==2:
        df['birth year']=year-df['birth year']
        df['birth year'] = df['birth year'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
    elif age==3:
        df['edad_usuario']=df['edad_usuario'].fillna(-99)
        df['edad_usuario'] = df['edad_usuario'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
    elif age==4:
        #pre 2018 q1
        #df['05 - Member Details Member Birthday Year']=year-df['05 - Member Details Member Birthday Year']
        #df['05 - Member Details Member Birthday Year']=df['05 - Member Details Member Birthday Year'].fillna(-99)
        #df['birthyear'] = df['05 - Member Details Member Birthday Year'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
        #else
        df['birthyear']=year-df['birthyear']
        df['birthyear']=df['birthyear'].fillna(-99)
        df['birthyear'] = df['birthyear'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
    elif age==5:
        df['Edad_Usuario'] = df['Edad_Usuario'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
    elif age==-9:
        df['Age'] = '-99'
    else:
        print('Formato no valido, calculo de edad no soportado')
        
        
    if usertype==1:
        df.user_type=df.user_type.apply(lambda y:1 if y=="2" else y)
        df.user_type=df.user_type.apply(lambda y:1 if y=="3" else y)
    elif usertype==2:
        #pre 2018 q1
        #df['usertype']=df['User Type'].apply(lambda y:1 if y=="Subscriber" else 0)
        #else
        df.usertype=df.usertype.apply(lambda y:1 if y=="Subscriber" else 0)
    elif usertype==-9:
        df['User Type'] = '-99'
    else:
        print('Formato no valido, calculo de duracion no soportado')
        
        
    #Madrid
    if city==10:
        df['City'] = 'Madrid'
        keep_col=['Start Dates','End Dates','Start Time','End Time','travel_time','ageRange','Gender','user_type','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
        
    #NYork
    elif city==20:
        df['City'] = 'NewYork'
        keep_col=['Start Dates','End Dates','Start Time','End Time','Duration','birth year','gender','usertype','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
        
    #Mexico
    elif city==30:
        df['City'] = 'Mexico'
        keep_col=['Start Dates','End Dates','Start Time','End Time','Duration','Edad_Usuario','Genero_Usuario','User Type','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
    
    #Seoul
    elif city==40:
        df['City'] = 'Seoul'
        keep_col=['Start Dates','End Dates','Start Time','End Time','Duration','Age','Gender','User Type','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
        
    #London
    elif city==50:
        df['City'] = 'London'
        keep_col=['Start Dates','End Dates','Start Time','End Time','Duration','Age','Gender','User Type','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
        
    #Buenos aires
    elif city==60:
        df['City'] = 'BuenosAires'
        keep_col=['Start Dates','End Dates','Start Time','End Time','Duration','Age','genero_usuario','User Type','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
    
    #Chicago
    elif city==70:
        df['City'] = 'Chicago'
        keep_col=['Start Dates','End Dates','Start Time','End Time','Duration','birthyear','gender','usertype','City']
        new_df=df[keep_col]
        new_df.columns=['SDate','EDate','STime','ETime','Duration','Age','Gender','User Type','City']
    
    else:
        print("Tipo de seleccion no encontrada")
        
    new_df.to_csv(destination, index=False)

    return(new_df)

In [17]:
#to_canon('./TFM/inputNY_17_1.csv')
#to_canon('./TFM/inputNY_17_2.csv')
#to_canon('./TFM/inputNY_17_3.csv')
#to_canon('./TFM/inputNY_17_4.csv')
#to_canon('./TFM/inputNY_17_5.csv')
#to_canon('./TFM/inputNY_17_6.csv')
#to_canon('./TFM/inputNY_17_7.csv')
to_canon('./TFM/inputNY_17_8.csv')
to_canon('./TFM/inputNY_17_9.csv')
to_canon('./TFM/inputNY_17_10.csv')
to_canon('./TFM/inputNY_17_11.csv')
to_canon('./TFM/inputNY_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
0,2017-12-01,2017-12-01,00:00:59,00:08:32,453,4,1,1,NewYork
1,2017-12-01,2017-12-01,00:04:42,00:07:21,159,3,1,1,NewYork
2,2017-12-01,2017-12-01,00:17:00,00:19:55,175,3,1,1,NewYork
3,2017-12-01,2017-12-01,00:50:22,00:57:57,455,4,1,1,NewYork
4,2017-12-01,2017-12-01,00:55:03,01:06:13,670,5,1,1,NewYork
...,...,...,...,...,...,...,...,...,...
889962,2017-12-31,2018-01-01,23:54:22,00:01:00,398,5,2,1,NewYork
889963,2017-12-31,2018-01-01,23:54:44,00:00:16,332,4,1,1,NewYork
889964,2017-12-31,2018-01-01,23:56:07,00:05:33,566,0,1,1,NewYork
889965,2017-12-31,2018-01-01,23:57:16,00:24:56,1660,4,2,1,NewYork


In [11]:
to_canon('./TFM/inputSeoul_17_1.csv')
to_canon('./TFM/inputSeoul_17_2.csv')
to_canon('./TFM/inputSeoul_17_3.csv')
to_canon('./TFM/inputSeoul_17_4.csv')
to_canon('./TFM/inputSeoul_17_5.csv')
to_canon('./TFM/inputSeoul_17_6.csv')
to_canon('./TFM/inputSeoul_17_7.csv')
to_canon('./TFM/inputSeoul_17_8.csv')
to_canon('./TFM/inputSeoul_17_9.csv')
to_canon('./TFM/inputSeoul_17_10.csv')
to_canon('./TFM/inputSeoul_17_11.csv')
to_canon('./TFM/inputSeoul_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
286261,2017-12-10,2017-12-10,00:00:01,00:20:01,1200,-99,-99,-99,Seoul
286262,2017-12-10,2017-12-10,00:00:03,00:29:56,1793,-99,-99,-99,Seoul
286263,2017-12-10,2017-12-10,00:00:05,00:32:40,1955,-99,-99,-99,Seoul
286264,2017-12-10,2017-12-10,00:00:07,00:39:08,2341,-99,-99,-99,Seoul
286265,2017-12-10,2017-12-10,00:00:08,00:20:56,1248,-99,-99,-99,Seoul
...,...,...,...,...,...,...,...,...,...
672618,2017-12-31,2018-01-01,23:58:55,00:09:39,644,-99,-99,-99,Seoul
672619,2017-12-31,2018-01-01,23:58:56,00:33:01,2045,-99,-99,-99,Seoul
672620,2017-12-31,2018-01-01,23:59:00,00:49:23,3023,-99,-99,-99,Seoul
672621,2017-12-31,2018-01-01,23:59:46,00:07:18,452,-99,-99,-99,Seoul


In [20]:
to_canon('./TFM/inputBA_17_1.csv')
to_canon('./TFM/inputBA_17_2.csv')
to_canon('./TFM/inputBA_17_3.csv')
to_canon('./TFM/inputBA_17_4.csv')
to_canon('./TFM/inputBA_17_5.csv')
to_canon('./TFM/inputBA_17_6.csv')
to_canon('./TFM/inputBA_17_7.csv')
to_canon('./TFM/inputBA_17_8.csv')
to_canon('./TFM/inputBA_17_9.csv')
to_canon('./TFM/inputBA_17_10.csv')
to_canon('./TFM/inputBA_17_11.csv')
to_canon('./TFM/inputBA_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City


In [13]:
to_canon('./TFM/inputMadrid_17_1.csv')
to_canon('./TFM/inputMadrid_17_2.csv')
to_canon('./TFM/inputMadrid_17_3.csv')
to_canon('./TFM/inputMadrid_17_4.csv')
to_canon('./TFM/inputMadrid_17_5.csv')
to_canon('./TFM/inputMadrid_17_6.csv')
to_canon('./TFM/inputMadrid_17_7.csv')
to_canon('./TFM/inputMadrid_17_8.csv')
to_canon('./TFM/inputMadrid_17_9.csv')
to_canon('./TFM/inputMadrid_17_10.csv')
to_canon('./TFM/inputMadrid_17_11.csv')
to_canon('./TFM/inputMadrid_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
0,2017-12-01,2017-12-01,00:00:00,00:00:00,352,4,-99,1,Madrid
1,2017-12-01,2017-12-01,00:00:00,00:00:00,263,4,-99,1,Madrid
2,2017-12-01,2017-12-01,00:00:00,00:00:00,346,0,-99,1,Madrid
3,2017-12-01,2017-12-01,00:00:00,00:00:00,335,4,-99,1,Madrid
4,2017-12-01,2017-12-01,00:00:00,00:00:00,454,4,-99,1,Madrid
...,...,...,...,...,...,...,...,...,...
259709,2017-12-31,2017-12-31,23:00:00,23:00:00,711,0,-99,1,Madrid
259710,2017-12-31,2017-12-31,23:00:00,23:00:00,350,4,-99,1,Madrid
259711,2017-12-31,2017-12-31,23:00:00,23:00:00,3122,3,-99,1,Madrid
259712,2017-12-31,2017-12-31,23:00:00,23:00:00,4219,0,-99,1,Madrid


In [6]:
to_canon('./TFM/inputMex_17_1.csv')
to_canon('./TFM/inputMex_17_2.csv')
to_canon('./TFM/inputMex_17_3.csv')
to_canon('./TFM/inputMex_17_4.csv')
to_canon('./TFM/inputMex_17_5.csv')
to_canon('./TFM/inputMex_17_6.csv')
to_canon('./TFM/inputMex_17_7.csv')
to_canon('./TFM/inputMex_17_8.csv')
to_canon('./TFM/inputMex_17_9.csv')
to_canon('./TFM/inputMex_17_10.csv')
to_canon('./TFM/inputMex_17_11.csv')
to_canon('./TFM/inputMex_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
0,2017-12-01,2017-12-01,00:00:21,00:11:10,649,4,1,-99,Mexico
1,2017-12-01,2017-12-01,00:01:00,00:37:29,2189,5,2,-99,Mexico
2,2017-12-01,2017-12-01,00:01:58,00:12:45,647,4,2,-99,Mexico
3,2017-12-01,2017-12-01,00:02:21,00:04:01,100,4,2,-99,Mexico
4,2017-12-01,2017-12-01,00:03:48,00:06:35,167,0,1,-99,Mexico
...,...,...,...,...,...,...,...,...,...
630597,2017-12-30,2017-12-31,23:57:03,00:32:55,2152,4,1,-99,Mexico
630598,2017-12-30,2017-12-31,23:57:20,00:03:12,352,3,1,-99,Mexico
630599,2017-12-30,2017-12-31,23:57:47,00:00:00,133,4,1,-99,Mexico
630600,2017-12-30,2017-12-31,23:58:01,00:02:50,289,4,1,-99,Mexico


In [24]:
#to_canon('./TFM/inputLond_17_1.csv')
#to_canon('./TFM/inputLond_17_2.csv')
#to_canon('./TFM/inputLond_17_3.csv')
#to_canon('./TFM/inputLond_17_4.csv')
#to_canon('./TFM/inputLond_17_5.csv')
#to_canon('./TFM/inputLond_17_6.csv')
#to_canon('./TFM/inputLond_17_7.csv')
#to_canon('./TFM/inputLond_17_8.csv')
to_canon('./TFM/inputLond_17_9.csv')
to_canon('./TFM/inputLond_17_10.csv')
to_canon('./TFM/inputLond_17_11.csv')
to_canon('./TFM/inputLond_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
0,2017-12-02,2017-12-02,08:37:00,08:42:00,300,-99,-99,-99,London
1,2017-12-05,2017-12-05,06:38:00,06:52:00,840,-99,-99,-99,London
4,2017-12-03,2017-12-03,12:13:00,12:17:00,240,-99,-99,-99,London
5,2017-12-04,2017-12-04,07:10:00,07:27:00,1020,-99,-99,-99,London
6,2017-12-05,2017-12-05,08:41:00,08:59:00,1080,-99,-99,-99,London
...,...,...,...,...,...,...,...,...,...
68823,2017-12-29,2017-12-29,12:43:00,13:23:00,2400,-99,-99,-99,London
68824,2017-12-29,2017-12-29,12:47:00,12:59:00,720,-99,-99,-99,London
68825,2017-12-29,2017-12-29,12:26:00,13:14:00,2880,-99,-99,-99,London
68826,2017-12-30,2017-12-30,13:46:00,14:27:00,2460,-99,-99,-99,London


In [3]:
to_canon('./TFM/inputChicago_17_1.csv')
to_canon('./TFM/inputChicago_17_2.csv')
to_canon('./TFM/inputChicago_17_3.csv')
to_canon('./TFM/inputChicago_17_4.csv')
to_canon('./TFM/inputChicago_17_5.csv')
to_canon('./TFM/inputChicago_17_6.csv')
to_canon('./TFM/inputChicago_17_7.csv')
to_canon('./TFM/inputChicago_17_8.csv')
to_canon('./TFM/inputChicago_17_9.csv')
to_canon('./TFM/inputChicago_17_10.csv')
to_canon('./TFM/inputChicago_17_11.csv')
to_canon('./TFM/inputChicago_17_12.csv')

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
0,2017-12-31,2018-01-01,23:58:00,00:03:00,300,4,1.0,1,Chicago
1,2017-12-31,2018-01-01,23:54:00,00:18:00,1440,0,-99.0,0,Chicago
2,2017-12-31,2018-01-01,23:54:00,00:18:00,1440,0,-99.0,0,Chicago
3,2017-12-31,2017-12-31,23:48:00,23:53:00,300,5,1.0,1,Chicago
4,2017-12-31,2017-12-31,23:42:00,23:47:00,300,0,1.0,1,Chicago
...,...,...,...,...,...,...,...,...,...
125391,2017-12-01,2017-12-01,00:04:00,00:21:00,1020,4,1.0,1,Chicago
125392,2017-12-01,2017-12-01,00:03:00,00:29:00,1560,0,2.0,1,Chicago
125393,2017-12-01,2017-12-01,00:03:00,00:18:00,900,4,2.0,1,Chicago
125394,2017-12-01,2017-12-01,00:00:00,00:06:00,360,3,1.0,1,Chicago


In [25]:
sources=['./TFM/MData/BuenosAires2017_12.csv',
             './TFM/MData/Chicago2017_12.csv',
                 './TFM/MData/Lon2017_12.csv',
              './TFM/MData/Madrid2017_12.csv',
                 './TFM/MData/Mex2017_12.csv',
               './TFM/MData/NYork2017_12.csv',
               './TFM/MData/Seoul2017_12.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_12.csv', index=False)

In [26]:
sources=['./TFM/MData/BuenosAires2017_11.csv',
             './TFM/MData/Chicago2017_11.csv',
                 './TFM/MData/Lon2017_11.csv',
              './TFM/MData/Madrid2017_11.csv',
                 './TFM/MData/Mex2017_11.csv',
               './TFM/MData/NYork2017_11.csv',
               './TFM/MData/Seoul2017_11.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_11.csv', index=False)

In [27]:
sources=['./TFM/MData/BuenosAires2017_10.csv',
             './TFM/MData/Chicago2017_10.csv',
                 './TFM/MData/Lon2017_10.csv',
              './TFM/MData/Madrid2017_10.csv',
                 './TFM/MData/Mex2017_10.csv',
               './TFM/MData/NYork2017_10.csv',
               './TFM/MData/Seoul2017_10.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_10.csv', index=False)

In [28]:
sources=['./TFM/MData/BuenosAires2017_09.csv',
             './TFM/MData/Chicago2017_09.csv',
                 './TFM/MData/Lon2017_09.csv',
              './TFM/MData/Madrid2017_09.csv',
                 './TFM/MData/Mex2017_09.csv',
               './TFM/MData/NYork2017_09.csv',
               './TFM/MData/Seoul2017_09.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_09.csv', index=False)

In [29]:
sources=['./TFM/MData/BuenosAires2017_08.csv',
             './TFM/MData/Chicago2017_08.csv',
                 './TFM/MData/Lon2017_08.csv',
              './TFM/MData/Madrid2017_08.csv',
                 './TFM/MData/Mex2017_08.csv',
               './TFM/MData/NYork2017_08.csv',
               './TFM/MData/Seoul2017_08.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_08.csv', index=False)

In [30]:
sources=['./TFM/MData/BuenosAires2017_07.csv',
             './TFM/MData/Chicago2017_07.csv',
                 './TFM/MData/Lon2017_07.csv',
              './TFM/MData/Madrid2017_07.csv',
                 './TFM/MData/Mex2017_07.csv',
               './TFM/MData/NYork2017_07.csv',
               './TFM/MData/Seoul2017_07.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_07.csv', index=False)

In [31]:
sources=['./TFM/MData/BuenosAires2017_06.csv',
             './TFM/MData/Chicago2017_06.csv',
                 './TFM/MData/Lon2017_06.csv',
              './TFM/MData/Madrid2017_06.csv',
                 './TFM/MData/Mex2017_06.csv',
               './TFM/MData/NYork2017_06.csv',
               './TFM/MData/Seoul2017_06.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_06.csv', index=False)

In [32]:
sources=['./TFM/MData/BuenosAires2017_05.csv',
             './TFM/MData/Chicago2017_05.csv',
                 './TFM/MData/Lon2017_05.csv',
              './TFM/MData/Madrid2017_05.csv',
                 './TFM/MData/Mex2017_05.csv',
               './TFM/MData/NYork2017_05.csv',
               './TFM/MData/Seoul2017_05.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_05.csv', index=False)

In [33]:
sources=['./TFM/MData/BuenosAires2017_04.csv',
             './TFM/MData/Chicago2017_04.csv',
                 './TFM/MData/Lon2017_04.csv',
              './TFM/MData/Madrid2017_04.csv',
                 './TFM/MData/Mex2017_04.csv',
               './TFM/MData/NYork2017_04.csv',
               './TFM/MData/Seoul2017_04.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_04.csv', index=False)

In [34]:
sources=['./TFM/MData/BuenosAires2017_03.csv',
             './TFM/MData/Chicago2017_03.csv',
                 './TFM/MData/Lon2017_03.csv',
              './TFM/MData/Madrid2017_03.csv',
                 './TFM/MData/Mex2017_03.csv',
               './TFM/MData/NYork2017_03.csv',
               './TFM/MData/Seoul2017_03.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_03.csv', index=False)

In [35]:
sources=['./TFM/MData/BuenosAires2017_02.csv',
             './TFM/MData/Chicago2017_02.csv',
                 './TFM/MData/Lon2017_02.csv',
              './TFM/MData/Madrid2017_02.csv',
                 './TFM/MData/Mex2017_02.csv',
               './TFM/MData/NYork2017_02.csv',
               './TFM/MData/Seoul2017_02.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_02.csv', index=False)

In [36]:
sources=['./TFM/MData/BuenosAires2017_01.csv',
             './TFM/MData/Chicago2017_01.csv',
                 './TFM/MData/Lon2017_01.csv',
              './TFM/MData/Madrid2017_01.csv',
                 './TFM/MData/Mex2017_01.csv',
               './TFM/MData/NYork2017_01.csv',
               './TFM/MData/Seoul2017_01.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2017_01.csv', index=False)

In [ ]:
sources=['./TFM/FData/2019_01.csv',
         './TFM/FData/2019_02.csv',
         './TFM/FData/2019_03.csv',
         './TFM/FData/2019_04.csv',
         './TFM/FData/2019_05.csv',
         './TFM/FData/2019_06.csv',
         './TFM/FData/2019_07.csv',
         './TFM/FData/2019_08.csv',
         './TFM/FData/2019_09.csv',
         './TFM/FData/2019_10.csv',
         './TFM/FData/2019_11.csv',
         './TFM/FData/2019_12.csv']

df=pd.DataFrame()
for i in range(len(sources)):
    dfaux=pd.read_csv(sources[i],encoding='unicode_escape',low_memory=False)
    df=df.append(dfaux)

df.to_csv('./TFM/FData/2019_FY.csv', index=False)

In [58]:
df

,SDate,EDate,STime,ETime,Duration,Age,Gender,User Type,City
0,2018-04-01,2018-04-01,00:01:27,00:19:19,1072,0,1.0,0,BuenosAires
1,2018-04-01,2018-04-01,00:01:50,00:11:11,561,4,1.0,1,BuenosAires
2,2018-04-01,2018-04-01,00:02:04,00:25:04,1380,3,1.0,1,BuenosAires
3,2018-04-01,2018-04-01,00:02:22,01:08:29,3967,0,1.0,0,BuenosAires
4,2018-04-01,2018-04-01,00:02:32,00:11:56,564,3,2.0,1,BuenosAires
...,...,...,...,...,...,...,...,...,...
677873,2018-04-30,2018-05-01,23:59:49,00:11:58,729,-99,-99.0,-99,Seoul
677874,2018-04-30,2018-05-01,23:59:49,00:24:27,1478,-99,-99.0,-99,Seoul
677875,2018-04-30,2018-05-01,23:59:50,00:24:39,1489,-99,-99.0,-99,Seoul
677876,2018-04-30,2018-05-01,23:59:53,00:12:19,746,-99,-99.0,-99,Seoul


In [59]:
df.to_csv('./TFM/FData/2018_04.csv', index=False)

In [15]:
df=pd.read_json('./TFM/bdata/2018/201812_Usage_Bicimad.json', encoding='latin-1',lines=True)

In [16]:
df

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,track
0,{'$oid': '5c085aae2f384324b8a10925'},39d847f3af0e9d8ab5f29fa7b5af3612c87e84f4d35a1c...,7,3,7,16,75,5,75,{'$date': '2018-12-01T00:00:00.000+0100'},,NaN
1,{'$oid': '5c085aae2f384324b8a10928'},697479f320b109464122ad3ce33eb368a6bc6755682f95...,19,1,14,262,57,4,128,{'$date': '2018-12-01T00:00:00.000+0100'},28045,"{'type': 'FeatureCollection', 'features': [{'g..."
2,{'$oid': '5c085aae2f384324b8a1092c'},f71720d1858187177c256bc3c2ac2ad3f35822e26abc74...,22,1,17,325,90,0,77,{'$date': '2018-12-01T00:00:00.000+0100'},,NaN
3,{'$oid': '5c085aae2f384324b8a10931'},d0999b551f278b8ba5dffb41135202b76051566723774b...,20,1,2,386,60,3,175,{'$date': '2018-12-01T00:00:00.000+0100'},28020,"{'type': 'FeatureCollection', 'features': [{'g..."
4,{'$oid': '5c085aae2f384324b8a10937'},ee183a2c78afb690815158bf5802bba83d50f5c509599a...,15,1,11,259,46,0,129,{'$date': '2018-12-01T00:00:00.000+0100'},,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
273212,{'$oid': '5c4b07ec2f38432e007dac9b'},3a23d5f752018f783666f18ce4d4f6f1c8c403283c720e...,5,1,16,597,131,4,99,{'$date': '2019-01-01T00:00:00.000+0100'},28005,"{'type': 'FeatureCollection', 'features': [{'g..."
273213,{'$oid': '5c4b07ed2f38432e007dad00'},2db18c679c83b4d4d8a1e396289513453bc7a25f2077e4...,3,1,17,6365,49,0,12,{'$date': '2019-01-01T00:00:00.000+0100'},,"{'type': 'FeatureCollection', 'features': [{'g..."
273214,{'$oid': '5c4b07ed2f38432e007dad30'},ed299327498201ba27b3af77fda2b717378a53e52a2b97...,24,1,21,9860,136,0,50,{'$date': '2019-01-01T00:00:00.000+0100'},,"{'type': 'FeatureCollection', 'features': [{'g..."
273215,{'$oid': '5c4b07ed2f38432e007dad5f'},06518d15c01766b17dfcec2459f81a3e3fdc7418b532fb...,7,1,16,8581,164,0,84,{'$date': '2019-01-01T00:00:00.000+0100'},,"{'type': 'FeatureCollection', 'features': [{'g..."


In [97]:
df['starttime']=pd.to_datetime(df[df.columns[1]],errors='coerce')

In [98]:
df

,ýpý8,ý|ý,ýýý8,ýýýý,ýpX,ï¿½,starttime
0,SPB-01160,03/07/2018 06:39,2247,1,ýýýýý,03/07/2018 06:52,2018-03-07 06:39:00
1,SPB-07518,03/07/2018 20:30,2247,1,ýýýýý,03/07/2018 21:52,2018-03-07 20:30:00
2,SPB-16262,03/07/2018 20:29,2247,1,ýýýýý,03/07/2018 21:52,2018-03-07 20:29:00
3,SPB-07518,04/07/2018 19:26,2247,1,ýýýýý,04/07/2018 20:24,2018-04-07 19:26:00
4,SPB-12219,04/07/2018 21:23,2247,1,ýýýýý,04/07/2018 21:39,2018-04-07 21:23:00
...,...,...,...,...,...,...,...
1039440,SPB-18450,30/07/2018 20:09,417,1,DMCï¿½ï¿½ï¿,30/07/2018 20:29,2018-07-30 20:09:00
1039441,SPB-16947,30/07/2018 18:58,923,1,mï¿½ï¿½ï¿,30/07/2018 20:30,2018-07-30 18:58:00
1039442,SPB-16310,30/07/2018 20:18,1908,1,tï¿½ï¿½ï¿½ï,30/07/2018 20:30,2018-07-30 20:18:00
1039443,SPB-15141,30/07/2018 20:13,2324,1,ýýP X 2ý Pý ,30/07/2018 20:30,2018-07-30 20:13:00


In [24]:
df=pd.read_csv('./TFM/bdata/2018/Divvy_Trips_2018_Q1.csv',encoding='unicode_escape',low_memory=False)
df

,01 - Rental Details Rental ID,01 - Rental Details Local Start Time,01 - Rental Details Local End Time,01 - Rental Details Bike ID,01 - Rental Details Duration In Seconds Uncapped,03 - Rental Start Station ID,03 - Rental Start Station Name,02 - Rental End Station ID,02 - Rental End Station Name,User Type,Member Gender,05 - Member Details Member Birthday Year
0,17536702,2018-01-01 00:12:00,2018-01-01 00:17:23,3304,323.0,69,Damen Ave & Pierce Ave,159,Claremont Ave & Hirsch St,Subscriber,Male,1988.0
1,17536703,2018-01-01 00:41:35,2018-01-01 00:47:52,5367,377.0,253,Winthrop Ave & Lawrence Ave,325,Clark St & Winnemac Ave (Temp),Subscriber,Male,1984.0
2,17536704,2018-01-01 00:44:46,2018-01-01 01:33:10,4599,"2,904.0",98,LaSalle St & Washington St,509,Troy St & North Ave,Subscriber,Male,1989.0
3,17536705,2018-01-01 00:53:10,2018-01-01 01:05:37,2302,747.0,125,Rush St & Hubbard St,364,Larrabee St & Oak St,Subscriber,Male,1983.0
4,17536706,2018-01-01 00:53:37,2018-01-01 00:56:40,3696,183.0,129,Blue Island Ave & 18th St,205,Paulina St & 18th St,Subscriber,Male,1989.0
...,...,...,...,...,...,...,...,...,...,...,...,...
387140,18000522,2018-03-31 23:46:34,2018-04-01 00:05:24,1935,"1,130.0",158,Milwaukee Ave & Wabansia Ave,260,Kedzie Ave & Milwaukee Ave,Subscriber,Male,1982.0
387141,18000523,2018-03-31 23:47:43,2018-03-31 23:52:05,5852,262.0,299,Halsted St & Roscoe St,229,Southport Ave & Roscoe St,Subscriber,Male,1989.0
387142,18000524,2018-03-31 23:50:18,2018-03-31 23:57:38,4414,440.0,327,Sheffield Ave & Webster Ave,226,Racine Ave & Belmont Ave,Subscriber,Male,1991.0
387143,18000525,2018-03-31 23:52:26,2018-04-01 00:07:13,6448,887.0,265,Cottage Grove Ave & Oakwood Blvd,426,Ellis Ave & 60th St,Subscriber,Male,1991.0


In [ ]:
'ÀÚÀü°Å¹øÈ£','´ë¿©ÀÏ½Ã','´ë¿©´ë¿©¼Ò¹øÈ£','´ë¿©´ë¿©¼Ò¸í','´ë¿©°ÅÄ¡´ë','¹Ý³³ÀÏ½Ã','¹Ý³³´ë¿©¼Ò¹øÈ£','¹Ý³³´ë¿©¼Ò¸í','¹Ý³³°ÅÄ¡´ë','ÀÌ¿ë½Ã°£(ºÐ)','ÀÌ¿ë°Å¸®(M)'

In [92]:
df=pd.read_csv('./TFM/BData/2018/seoul_201807_1M.csv',encoding='unicode_escape',low_memory=False)
df

,ýpý8,ý|ý,ýýý8,ýýýý,ýpX,ï¿½
0,SPB-01160,03/07/2018 06:39,2247,1,ýýýýý,03/07/2018 06:52
1,SPB-07518,03/07/2018 20:30,2247,1,ýýýýý,03/07/2018 21:52
2,SPB-16262,03/07/2018 20:29,2247,1,ýýýýý,03/07/2018 21:52
3,SPB-07518,04/07/2018 19:26,2247,1,ýýýýý,04/07/2018 20:24
4,SPB-12219,04/07/2018 21:23,2247,1,ýýýýý,04/07/2018 21:39
...,...,...,...,...,...,...
1039440,SPB-18450,30/07/2018 20:09,417,1,DMCï¿½ï¿½ï¿,30/07/2018 20:29
1039441,SPB-16947,30/07/2018 18:58,923,1,mï¿½ï¿½ï¿,30/07/2018 20:30
1039442,SPB-16310,30/07/2018 20:18,1908,1,tï¿½ï¿½ï¿½ï,30/07/2018 20:30
1039443,SPB-15141,30/07/2018 20:13,2324,1,ýýP X 2ý Pý ,30/07/2018 20:30


In [87]:
df.dtypes

ýpý8    object
ý|ý      object
ýýý8     object
ýýýý     object
ýpX      object
ï¿½     object
dtype: object

In [57]:
df[df.columns[5]]

0         '2018-01-01 00:04:03'
1         '2018-01-01 00:25:41'
2         '2018-01-01 00:09:33'
3         '2018-01-01 00:10:13'
4         '2018-01-01 00:06:18'
                  ...          
784729    '2018-04-01 00:26:50'
784730    '2018-04-01 00:07:30'
784731    '2018-04-01 00:40:21'
784732    '2018-04-01 00:04:38'
784733    '2018-04-01 01:45:18'
Name: '©öY©ø©øAI¨öA', Length: 784734, dtype: object

In [ ]:
df4=pd.read_csv('./TFM/BData/seoul_201904.csv',low_memory=False)
df4

In [270]:
df4=df4.sort_values('¹Ý³³ÀÏ½Ã')
dfsin=df4.iloc[14:]

In [271]:
dfsin

,ÀÚÀü°Å¹øÈ£,´ë¿©ÀÏ½Ã,´ë¿©¼Ò¹øÈ£,´ë¿©´ë¿©¼Ò¸í,°ÅÄ¡´ë¼ø¹ø,¹Ý³³ÀÏ½Ã,´ë¿©¼Ò¹øÈ£.1,¹Ý³³´ë¿©¼Ò¸í,¹Ý³³°ÅÄ¡´ë¼ø¹ø,ÀÌ¿ë½Ã°£,ÀÌ¿ë°Å¸®
0,SPB-00352,2019-04-01 00:00:07,364,Ã¢½Å¿ª 1¹øÃâ±¸ ¾Õ,8,2019-04-01 00:04:36,00361,µ¿¹¦¾Õ¿ª 1¹øÃâ±¸ µÚ,4,4.0,740.0
1,SPB-11337,2019-04-01 00:02:10,1449,»óºÀ¿ª 1¹øÃâ±¸,5,2019-04-01 00:06:03,01404,µ¿ÀÏ·Î ÁöÇÏÂ÷µµ,10,3.0,550.0
2,SPB-05337,2019-04-01 00:02:24,1044,±ÁÀº´Ù¸®¿ª,12,2019-04-01 00:06:06,01015,»û¸¶À» ±Ù¸°°ø¿ø,13,3.0,880.0
3,SPB-17568,2019-04-01 00:02:15,127,Çö´ëº¥Ã³ºô ¾Õ,1,2019-04-01 00:06:29,00437,´ëÈï¿ª 1¹øÃâ±¸,5,3.0,760.0
4,SPB-12144,2019-04-01 00:01:57,1906,½Åµµ¸²¿ª 1¹ø Ãâ±¸ ¾Õ,1,2019-04-01 00:07:16,00263,±Ù·ÎÀÚÈ¸°ü »ç°Å¸®,2,5.0,1100.0
...,...,...,...,...,...,...,...,...,...,...,...
1427762,SPB-08266,2019-04-30 23:59:49,1016,ÇØ¶ß´Â ÁÖÀ¯¼Ò¿· ¸®¿£ÆÄÅ© 109µ¿¾Õ,6,2019-05-01 04:16:22,01014,°­µ¿±¸Æò»ýÇÐ½À°ü¾Õ,13,255.0,3480.0
1427763,SPB-11986,2019-04-30 21:40:14,906,¿¬½Å³»¿ª 5¹øÃâ±¸150M ¾Æ·¡,3,2019-05-01 05:58:26,00930,±¸ ¼­ºÎ°æÂû¼­ °Ç³ÊÆí,5,242.0,2560.0
1427764,SPB-12383,2019-04-30 23:31:31,1824,µ¶»ê±Ù¸°°ø¿ø ÀÔ±¸,6,2019-05-01 07:46:10,01824,µ¶»ê±Ù¸°°ø¿ø ÀÔ±¸,6,236.0,11590.0
1427765,SPB-10614,2019-04-30 21:53:10,421,¸¶Æ÷±¸Ã» ¾Õ,6,2019-05-01 08:51:17,00402,»ó¾Ï¿ùµåÄÅÆÄÅ© 9´ÜÁö ¾Õ,3,140.0,3890.0


In [272]:
dfsin.to_csv('./TFM/BData/seoul_201904.csv', index=False,encoding='unicode_escape')

In [275]:
dfsin

,ÀÚÀü°Å¹øÈ£,´ë¿©ÀÏ½Ã,´ë¿©¼Ò¹øÈ£,´ë¿©´ë¿©¼Ò¸í,°ÅÄ¡´ë¼ø¹ø,¹Ý³³ÀÏ½Ã,´ë¿©¼Ò¹øÈ£.1,¹Ý³³´ë¿©¼Ò¸í,¹Ý³³°ÅÄ¡´ë¼ø¹ø,ÀÌ¿ë½Ã°£,ÀÌ¿ë°Å¸®
220026,SPB-09764,2019-04-06 13:15:31,2180,½Å¸²µ¿ÁÖ¹Î¼¾ÅÍ,7,2019-04-06 13:19:35,02135,½Å¸²¿ª 5¹øÃâ±¸,6,3.0,520.0
220027,SPB-03355,2019-04-06 11:55:50,207,¿©ÀÇ³ª·ç¿ª 1¹øÃâ±¸ ¾Õ,5,2019-04-06 13:19:40,00207,¿©ÀÇ³ª·ç¿ª 1¹øÃâ±¸ ¾Õ,3,83.0,6250.0
220028,SPB-13972,2019-04-06 13:02:13,1158,°¡¾ç¿ª 8¹øÃâ±¸,11,2019-04-06 13:19:44,01160,¾çÃµÇâ±³¿ª 7¹øÃâ±¸¾Õ,3,17.0,1670.0
220029,SPB-01963,2019-04-06 13:09:28,1233,Àá½Ç3°Å¸®(°¶·¯¸®¾ÆÆÓ¸®½º),7,2019-04-06 13:19:45,01211,¹æÀÌ»ï°Å¸®,1,9.0,1690.0


In [259]:
df4smol=df4[:5]
df4

,llegada
0,00114
1,00162
2,00230
3,00232
4,00374
...,...
1427762,2019-05-01 04:16:22
1427763,2019-05-01 05:58:26
1427764,2019-05-01 07:46:10
1427765,2019-05-01 08:51:17


In [197]:
keep_col=['´ë¿©ÀÏ½Ã','¹Ý³³ÀÏ½Ã']
new_df=df4smol[keep_col]
new_df.columns=['´ë¿©ÀÏ½Ã','¹Ý³³ÀÏ½Ã']

In [203]:
new_df

,´ë¿©ÀÏ½Ã,¹Ý³³ÀÏ½Ã,starttime,stoptime
0,2019-04-01 00:00:07,2019-04-01 00:04:36,2019-04-01 00:00:07,2019-04-01 00:04:36
1,2019-04-01 00:02:10,2019-04-01 00:06:03,2019-04-01 00:02:10,2019-04-01 00:06:03
2,2019-04-01 00:02:24,2019-04-01 00:06:06,2019-04-01 00:02:24,2019-04-01 00:06:06
3,2019-04-01 00:02:15,2019-04-01 00:06:29,2019-04-01 00:02:15,2019-04-01 00:06:29
4,2019-04-01 00:01:57,2019-04-01 00:07:16,2019-04-01 00:01:57,2019-04-01 00:07:16


In [201]:
new_df['starttime']=pd.to_datetime(new_df['´ë¿©ÀÏ½Ã'])
new_df['stoptime']=pd.to_datetime(new_df['¹Ý³³ÀÏ½Ã'])

C:\Users\UO231627\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\UO231627\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [202]:
new_df.dtypes

´ë¿©ÀÏ½Ã             object
¹Ý³³ÀÏ½Ã             object
starttime    datetime64[ns]
stoptime     datetime64[ns]
dtype: object

In [93]:
df3['starttime']=pd.to_datetime(df3['´ë¿©ÀÏ½Ã'], dayfirst = True)
df3['stoptime']=pd.to_datetime(df3['¹Ý³³ÀÏ½Ã'], dayfirst = True)

In [94]:
df3

,ÀÚÀü°Å¹øÈ£,´ë¿©ÀÏ½Ã,´ë¿©¼Ò¹øÈ£,´ë¿©´ë¿©¼Ò¸í,°ÅÄ¡´ë¼ø¹ø,¹Ý³³ÀÏ½Ã,´ë¿©¼Ò¹øÈ£.1,¹Ý³³´ë¿©¼Ò¸í,¹Ý³³°ÅÄ¡´ë¼ø¹ø,ÀÌ¿ë½Ã°£,ÀÌ¿ë°Å¸®,starttime,stoptime
0,SPB-13846,2019-03-01 00:02:45,1060,1060. ÃµÀÏÃÊ±³ »ç°Å¸®,1,2019-03-01 00:04:58,1061,1060. ÃµÀÏÃÊ±³ »ç°Å¸®,1,1,310.0,2019-03-01 00:02:45,2019-03-01 00:04:58
1,SPB-18088,2019-03-01 00:00:34,284,284. ¼¾Æ®·² Çª¸£Áö¿À ½ÃÆ¼ ¾Õ,8,2019-03-01 00:05:03,239,284. ¼¾Æ®·² Çª¸£Áö¿À ½ÃÆ¼ ¾Õ,1,4,970.0,2019-03-01 00:00:34,2019-03-01 00:05:03
2,SPB-04107,2019-03-01 00:01:37,262,262. ¿µ¹®ÃÊµîÇÐ±³ »ç°Å¸®,4,2019-03-01 00:05:10,275,262. ¿µ¹®ÃÊµîÇÐ±³ »ç°Å¸®,1,3,670.0,2019-03-01 00:01:37,2019-03-01 00:05:10
3,SPB-05969,2019-03-01 00:00:36,126,126. ¼­°­´ë ÈÄ¹® ¿·,5,2019-03-01 00:05:31,122,126. ¼­°­´ë ÈÄ¹® ¿·,7,4,740.0,2019-03-01 00:00:36,2019-03-01 00:05:31
4,SPB-18300,2019-03-01 00:01:31,2127,2127. º¸¼º¿î¼öÂ÷°íÁö ¸ÂÀºÆí,6,2019-03-01 00:05:48,2106,2127. º¸¼º¿î¼öÂ÷°íÁö ¸ÂÀºÆí,11,3,1090.0,2019-03-01 00:01:31,2019-03-01 00:05:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875241,SPB-14576,2019-03-31 21:31:33,1249,1249. ¾ÆÁÖÁßÇÐ±³°Ç³ÊÆí,7,2019-04-01 02:41:52,2619,1249. ¾ÆÁÖÁßÇÐ±³°Ç³ÊÆí,8,309,6180.0,2019-03-31 21:31:33,2019-04-01 02:41:52
875242,SPB-05879,2019-03-31 21:50:16,1248,1248. ¹æÀÌÃÊµîÇÐ±³,11,2019-04-01 02:48:57,1248,1248. ¹æÀÌÃÊµîÇÐ±³,19,298,6840.0,2019-03-31 21:50:16,2019-04-01 02:48:57
875243,SPB-05239,2019-03-31 07:35:10,1734,1734. ½Ö¹®Çö´ë1Â÷¾ÆÆÄÆ® 108µ¿ ¾Õ,5,2019-04-01 03:48:42,1733,1734. ½Ö¹®Çö´ë1Â÷¾ÆÆÄÆ® 108µ¿ ¾Õ,1,1213,3900.0,2019-03-31 07:35:10,2019-04-01 03:48:42
875244,SPB-09219,2019-03-31 22:53:44,1011,1011. LIGA ¾ÆÆÄÆ® ¾Õ,4,2019-04-01 05:29:36,1020,1011. LIGA ¾ÆÆÄÆ® ¾Õ,10,0,0.0,2019-03-31 22:53:44,2019-04-01 05:29:36


In [240]:
#df4['starttime']=pd.to_datetime(df4['´ë¿©ÀÏ½Ã'], dayfirst = True)

df4['stoptime']=pd.to_datetime(df4['¹Ý³³ÀÏ½Ã'][101050:])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 633-01-01 00:00:00

In [249]:
df4=df4.sort_values('¹Ý³³ÀÏ½Ã')
df4

,ÀÚÀü°Å¹øÈ£,´ë¿©ÀÏ½Ã,´ë¿©¼Ò¹øÈ£,´ë¿©´ë¿©¼Ò¸í,°ÅÄ¡´ë¼ø¹ø,¹Ý³³ÀÏ½Ã,´ë¿©¼Ò¹øÈ£.1,¹Ý³³´ë¿©¼Ò¸í,¹Ý³³°ÅÄ¡´ë¼ø¹ø,ÀÌ¿ë½Ã°£,ÀÌ¿ë°Å¸®
1145030,SPB-14828,2019-04-25 08:44:07,129,"½ÅÃÌ¿ª(2È£¼±) 6¹øÃâ±¸ ??,7""",2019-04-25 08:51:22,00114,È«´ëÀÔ±¸¿ª 8¹øÃâ±¸ ¾Õ,3,6,1410.0,NaN
987250,SPB-08024,2019-04-22 16:53:03,137,"NH³óÇù ½ÅÃÌÁöÁ¡ ??,7""",2019-04-22 17:04:46,00162,ºÀ¿ø°í°¡Â÷µµ ¹Ø,14,11,1500.0,NaN
1236228,SPB-07381,2019-04-27 21:26:26,184,"SK¸Á¿øµ¿ÁÖÀ¯¼Ò ?Ç³ÊÆ?,7""",2019-04-27 21:55:38,00230,¿µµîÆ÷±¸Ã»¿ª 1¹øÃâ±¸,3,27,4540.0,NaN
839089,SPB-04676,2019-04-19 18:30:57,232,"¾çÆò¿ì¸² ÀÌºñÁî¼¾Å¸ ??,1""",2019-04-19 19:28:02,00232,¾çÆò¿ì¸² ÀÌºñÁî¼¾Å¸ ¾Õ,1,56,2400.0,NaN
400748,SPB-07145,2019-04-11 10:26:02,375,"´Ù»ê ¾î¸°ÀÌ°ø??,7""",2019-04-11 10:30:28,00374,Ã»±¸¿ª 2¹øÃâ±¸ ¾Õ,5,4,720.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1427762,SPB-08266,2019-04-30 23:59:49,1016,ÇØ¶ß´Â ÁÖÀ¯¼Ò¿· ¸®¿£ÆÄÅ© 109µ¿¾Õ,6,2019-05-01 04:16:22,01014,°­µ¿±¸Æò»ýÇÐ½À°ü¾Õ,13,255.0,3480.0
1427763,SPB-11986,2019-04-30 21:40:14,906,¿¬½Å³»¿ª 5¹øÃâ±¸150M ¾Æ·¡,3,2019-05-01 05:58:26,00930,±¸ ¼­ºÎ°æÂû¼­ °Ç³ÊÆí,5,242.0,2560.0
1427764,SPB-12383,2019-04-30 23:31:31,1824,µ¶»ê±Ù¸°°ø¿ø ÀÔ±¸,6,2019-05-01 07:46:10,01824,µ¶»ê±Ù¸°°ø¿ø ÀÔ±¸,6,236.0,11590.0
1427765,SPB-10614,2019-04-30 21:53:10,421,¸¶Æ÷±¸Ã» ¾Õ,6,2019-05-01 08:51:17,00402,»ó¾Ï¿ùµåÄÅÆÄÅ© 9´ÜÁö ¾Õ,3,140.0,3890.0


In [255]:
dfsin=df4[14:]
dfsin.to_csv('./TFM/BData/seoul_.csv', index=False)

,ÀÚÀü°Å¹øÈ£,´ë¿©ÀÏ½Ã,´ë¿©¼Ò¹øÈ£,´ë¿©´ë¿©¼Ò¸í,°ÅÄ¡´ë¼ø¹ø,¹Ý³³ÀÏ½Ã,´ë¿©¼Ò¹øÈ£.1,¹Ý³³´ë¿©¼Ò¸í,¹Ý³³°ÅÄ¡´ë¼ø¹ø,ÀÌ¿ë½Ã°£,ÀÌ¿ë°Å¸®
0,SPB-00352,2019-04-01 00:00:07,364,Ã¢½Å¿ª 1¹øÃâ±¸ ¾Õ,8,2019-04-01 00:04:36,00361,µ¿¹¦¾Õ¿ª 1¹øÃâ±¸ µÚ,4,4.0,740.0
1,SPB-11337,2019-04-01 00:02:10,1449,»óºÀ¿ª 1¹øÃâ±¸,5,2019-04-01 00:06:03,01404,µ¿ÀÏ·Î ÁöÇÏÂ÷µµ,10,3.0,550.0
2,SPB-05337,2019-04-01 00:02:24,1044,±ÁÀº´Ù¸®¿ª,12,2019-04-01 00:06:06,01015,»û¸¶À» ±Ù¸°°ø¿ø,13,3.0,880.0
3,SPB-17568,2019-04-01 00:02:15,127,Çö´ëº¥Ã³ºô ¾Õ,1,2019-04-01 00:06:29,00437,´ëÈï¿ª 1¹øÃâ±¸,5,3.0,760.0
4,SPB-12144,2019-04-01 00:01:57,1906,½Åµµ¸²¿ª 1¹ø Ãâ±¸ ¾Õ,1,2019-04-01 00:07:16,00263,±Ù·ÎÀÚÈ¸°ü »ç°Å¸®,2,5.0,1100.0
...,...,...,...,...,...,...,...,...,...,...,...
1427762,SPB-08266,2019-04-30 23:59:49,1016,ÇØ¶ß´Â ÁÖÀ¯¼Ò¿· ¸®¿£ÆÄÅ© 109µ¿¾Õ,6,2019-05-01 04:16:22,01014,°­µ¿±¸Æò»ýÇÐ½À°ü¾Õ,13,255.0,3480.0
1427763,SPB-11986,2019-04-30 21:40:14,906,¿¬½Å³»¿ª 5¹øÃâ±¸150M ¾Æ·¡,3,2019-05-01 05:58:26,00930,±¸ ¼­ºÎ°æÂû¼­ °Ç³ÊÆí,5,242.0,2560.0
1427764,SPB-12383,2019-04-30 23:31:31,1824,µ¶»ê±Ù¸°°ø¿ø ÀÔ±¸,6,2019-05-01 07:46:10,01824,µ¶»ê±Ù¸°°ø¿ø ÀÔ±¸,6,236.0,11590.0
1427765,SPB-10614,2019-04-30 21:53:10,421,¸¶Æ÷±¸Ã» ¾Õ,6,2019-05-01 08:51:17,00402,»ó¾Ï¿ùµåÄÅÆÄÅ© 9´ÜÁö ¾Õ,3,140.0,3890.0


In [206]:
#df4aux=df4[:1]
#print(df4aux)
#df4aux['stoptime']=pd.to_datetime(df4['¹Ý³³ÀÏ½Ã'], dayfirst = True,format='%Y-%m-%d %H:%M:%S')
#print(df4aux)
#print(df4aux.info())
print(type(df4aux['¹Ý³³ÀÏ½Ã'][0])

SyntaxError: unexpected EOF while parsing (<ipython-input-206-78db1b6d789c>, line 6)

In [256]:
keep_col=['¹Ý³³ÀÏ½Ã']
new_df=df4[keep_col]
new_df.columns=['llegada']
new_df.to_csv('./TFM/BData/seoul_201904.csv', index=False)

In [128]:
df4[keep_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427767 entries, 0 to 1427766
Data columns (total 1 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   ¹Ý³³ÀÏ½Ã  1427767 non-null  object
dtypes: object(1)
memory usage: 10.9+ MB


In [164]:
df4[keep_col].iloc[0]

¹Ý³³ÀÏ½Ã    2019-04-01 00:04:36
Name: 0, dtype: object